In [1]:
import pandas as pd
import numpy as np
import pywt
import matplotlib.pyplot as plt
from scipy import interpolate
import glob
from mpl_toolkits.mplot3d import Axes3D
from matplotlib import cm


import heapq
from scipy.signal import argrelextrema

from scipy import signal
from scipy.ndimage.interpolation import shift
import pywt
from numpy.fft import fft
from numpy import zeros, floor, log10, log, mean, array, sqrt, vstack, cumsum, ones, log2, std
from numpy.linalg import svd, lstsq
import time

from sklearn.decomposition import PCA
import time
from joblib import Parallel, delayed
import multiprocessing

from sklearn.model_selection import train_test_split
import lightgbm as lgb
from catboost import CatBoostRegressor, FeaturesData, Pool
from sklearn.metrics import mean_squared_error

/Users/akuma824/Desktop/anaconda3/lib/python3.7/site-packages/lightgbm/__init__.py:48: UserWarning: Starting from version 2.2.1, the library file in distribution wheels for macOS is built by the Apple Clang (Xcode_8.3.3) compiler.
This means that in case of installing LightGBM from PyPI via the ``pip install lightgbm`` command, you don't need to install the gcc compiler anymore.
Instead of that, you need to install the OpenMP library, which is required for running LightGBM on the system with the Apple Clang compiler.
You can install the OpenMP library by the following command: ``brew install libomp``.
  "You can install the OpenMP library by the following command: ``brew install libomp``.", UserWarning)


In [2]:
#==========================================================================================
#making time stamp uniform by Interpolation

def preprocess(data):
    freq=50
    ls=['X','Y','Z']
    t1=np.arange(data.Timestamp[0],data.Timestamp[(data.shape[0])-1],0.02)
    df=pd.DataFrame({'Timestamp':t1})
    for i in ls:
        fcubic = interpolate.interp1d(data.Timestamp, data[i], kind='cubic')
        df[i]=fcubic(t1)
    df.columns=['Timestamp','acc_X','acc_Y','acc_Z']
    return df



def preprocess_real_smartphone(data):
    data.rename(columns={'t':'Timestamp','x':'X','y':'Y','z':'Z'},inplace=True)
    ls=['X','Y','Z']
    freq=round((1/((data.Timestamp.max()/data.Timestamp.shape[0]).round(3))),0)
    t1=np.arange(data.Timestamp[0],data.Timestamp[(data.shape[0])-1],(data.Timestamp.max()/data.Timestamp.shape[0]).round(3))
    #t1=np.arange(data.Timestamp[0],data.Timestamp[(data.shape[0])-1],0.01)
    df=pd.DataFrame({'Timestamp':t1})
    for i in ls:
        fcubic = interpolate.interp1d(data.Timestamp, data[i], kind='cubic')
        df[i]=fcubic(t1)
    df.columns=['Timestamp','acc_X','acc_Y','acc_Z']
    return df,freq


#==========================================================================================
#median filter
from scipy.signal import medfilt # import the median filter function
def median(signal):# input: numpy array 1D (one column)  
    #applying the median filter
    return  medfilt(np.array(signal), kernel_size=3) # applying the median filter order3(kernel_size=3)


#==========================================================================================
#components_selection_one_signal
import math # import math library


def components_selection_one_signal(t_signal,sampling_freq):
    nyq=sampling_freq/float(2) # nyq is the nyquist frequency equal to the half of the sampling frequency[50/2= 25 Hz]

    freq1 = 0.3
    freq2 = 20

    t_signal=np.array(t_signal)
    t_signal_length=len(t_signal) # number of points in a t_signal
    
    # the t_signal in frequency domain after applying fft
    f_signal=np.fft.fft(t_signal) # 1D numpy array contains complex values (in C)
    
    # generate frequencies associated to f_signal complex values
    freqs=np.array(np.fft.fftfreq(t_signal_length, d=1/float(sampling_freq))) # frequency values between [-25hz:+25hz]
        
    df=pd.DataFrame({'freq':abs(freqs),'amplitute':f_signal})
    df['f_DC_signal']=np.where(df.freq>freq1,0,df.amplitute)
    df['f_noise_signal']=np.where(df.freq<=freq2,0,df.amplitute)
    df['f_body_signal']=np.where(df.freq<=freq1,0,np.where(df.freq>freq2,0,df.amplitute))

    
    # Inverse the transformation of signals in freq domain #
    # applying the inverse fft(ifft) to signals in freq domain and put them in float format
    t_DC_component= np.fft.ifft(np.array(df['f_DC_signal'])).real
    t_body_component= np.fft.ifft(np.array(df['f_body_signal'])).real
    t_noise=np.fft.ifft(np.array(df['f_noise_signal'])).real
    
    total_component=t_signal-t_noise # extracting the total component(filtered from noise) 
                                     #  by substracting noise from t_signal (the original signal).
    
    # return outputs mentioned earlier
    return (total_component,t_DC_component,t_body_component,t_noise) 


#=================================================================================================================
#Define verify gravity function
def mag_3_signals(df): # Euclidian magnitude
    return np.array(np.sqrt(np.square(df).sum(axis=1)))

def verify_gravity(data):
    
    acc_x=np.array(data['acc_X']) # copy acc_X column from dataframe in raw_dic having the key mentioned above
    acc_y=np.array(data['acc_Y'])# copy acc_Y column  from dataframe in raw_dic having the key mentioned above
    acc_z=np.array(data['acc_Z'])# copy acc_Z column  from dataframe in raw_dic having the key mentioned above

    # apply the filtering method to acc_[X,Y,Z] and store gravity components
    grav_acc_X=components_selection_one_signal(acc_x)[1] 
    grav_acc_Y=components_selection_one_signal(acc_y)[1]
    grav_acc_Z=components_selection_one_signal(acc_z)[1]
    
    # calculating gravity magnitude signal
    grav_acc_mag=mag_3_signals(grav_acc_X, grav_acc_Y,grav_acc_Z)
    print('mean value = ',round((sum(grav_acc_mag) / len(grav_acc_mag)),3),' g')
    
#=================================================================================================================    
#Define jerking and magnitude functions
def jerk_one_signal(signal,sampling_freq):
    signal=pd.DataFrame(signal)
    jerk=(signal.shift(-1)-signal)*sampling_freq
    return np.array(jerk.dropna()).transpose()[0]

def time_domain_signal(data,sampling_freq):
    time_sig_df=pd.DataFrame()
    for column in ['acc_X','acc_Y','acc_Z']:
        t_signal=np.array(data[column])
        #med_filtred=median(t_signal)
        med_filtred=(t_signal)
        _,grav_acc,body_acc,_=components_selection_one_signal(med_filtred,sampling_freq)
        body_acc_jerk=jerk_one_signal(body_acc,sampling_freq)
        time_sig_df['t_body_'+column]=body_acc[:-1]
        time_sig_df['t_grav_'+column]= grav_acc[:-1]
        time_sig_df['t_body_acc_jerk_'+column[-1]]=body_acc_jerk

    # all 15 axial signals generated above are reordered to facilitate magnitudes signals generation
    new_columns_ordered=['t_body_acc_X','t_body_acc_Y','t_body_acc_Z',
                              't_grav_acc_X','t_grav_acc_Y','t_grav_acc_Z',
                              't_body_acc_jerk_X','t_body_acc_jerk_Y','t_body_acc_jerk_Z']


    # create new dataframe to order columns
    time_sig_df=time_sig_df[new_columns_ordered]

    # Magnitude Features
    for i in range(0,9,3):
        mag_col_name=new_columns_ordered[i][:-1]+'mag'# Create the magnitude column name related to each 3-axial signals
        time_sig_df[mag_col_name]=mag_3_signals(time_sig_df[new_columns_ordered[i:i+3]]) # store the signal_mag with its appropriate column name

    return(time_sig_df)

In [3]:
a_real=glob.glob("training_data/smartphone_accelerometer/*.csv")

In [4]:
data,sampling_freq=preprocess_real_smartphone(pd.read_csv(a_real[14]))
print(data.shape)
data.head()

(239999, 4)


,Timestamp,acc_X,acc_Y,acc_Z
0,0.000,0.330400,0.486023,9.694122
1,0.005,0.308852,0.495600,9.703699
2,0.010,0.317626,0.496629,9.697804
3,0.015,0.313640,0.452504,9.679757
4,0.020,0.296881,0.474052,9.691729


In [5]:
time_sig_df=time_domain_signal(data,sampling_freq)
time_sig_df.shape

(239998, 12)

In [6]:
time_sig_df.head()

,t_body_acc_X,t_body_acc_Y,t_body_acc_Z,t_grav_acc_X,t_grav_acc_Y,t_grav_acc_Z,t_body_acc_jerk_X,t_body_acc_jerk_Y,t_body_acc_jerk_Z,t_body_acc_mag,t_grav_acc_mag,t_body_acc_jerk_mag
0,0.016403,0.015019,-0.008986,0.241659,0.438172,9.706851,5.394058,3.139477,-0.236475,0.023987,9.719740,6.245646
1,0.043373,0.030716,-0.010169,0.242028,0.438459,9.706896,4.246040,2.321064,-0.221879,0.054112,9.719807,4.844112
2,0.064603,0.042321,-0.011278,0.242397,0.438746,9.706941,2.622540,1.375334,-0.095120,0.078051,9.719875,2.962820
3,0.077716,0.049198,-0.011754,0.242766,0.439033,9.706986,0.880836,0.502243,0.091331,0.092727,9.719942,1.018067
4,0.082120,0.051709,-0.011297,0.243135,0.439320,9.707032,-0.618131,-0.141429,0.269791,0.097700,9.720009,0.689112


In [7]:
#Fractal Dimension using Katz FD algorithm
def katz(data):
    n = len(data)-1
    L = np.hypot(np.diff(data), 1).sum() # Sum of distances
    d = np.hypot(data - data[0], np.arange(len(data))).max() # furthest distance from first point
    return np.log10(n) / (np.log10(d/L) + np.log10(n))




#Coefficient of variation
def coeff_var(a):
    output = np.std(a)/np.mean(a) #computing coefficient of variation
    return output

#Mean and variance of Vertex to Vertex Slope
def slope(x):
    
    amp_max = np.array(x[argrelextrema(np.array(x), np.greater)[0]])
    t_max = argrelextrema(np.array(x), np.greater)[0]
    amp_min = np.array(x[argrelextrema(np.array(x), np.less)[0]])
    t_min = argrelextrema(np.array(x), np.less)[0]
    t = np.concatenate((t_max,t_min),axis=0)
    t.sort()#sort on the basis of time

    amp = np.zeros(len(t))
    res = np.zeros(len(t))
    
    for l in range(len(t)):
        amp[l]=x[t[l]]

    amp_diff = first_diff(amp)
    t_diff = first_diff(t)

    for q in range(len(amp_diff)):
        res[q] = amp_diff[q]/t_diff[q]         
    
    res=res[~np.isnan(res)]
    return [np.mean(res),np.std(res)] #returning mean and std of vertex to vertex slope


#Hjorth Parameter
def hjorth(input):                                              
    hfeatures = []
    diff_input = np.diff(input)
    diff_diffinput = np.diff(diff_input)
    
    hjorth_activity = np.var(input)
    hjorth_mobility = np.sqrt(np.var(diff_input)/hjorth_activity)
    hjorth_diffmobility = np.sqrt(np.var(diff_diffinput)/np.var(diff_input))
    hjorth_complexity = hjorth_diffmobility/hjorth_mobility
    
    hfeatures.append(hjorth_activity)
    hfeatures.append(hjorth_mobility)
    hfeatures.append(hjorth_complexity)
    
    return hfeatures  #returning hjorth activity, hjorth mobility , hjorth complexity


#Kurtosis
def kurtosis(a):
    mean_i = np.mean(a) # Saving the mean of array i
    std_i = np.std(a) # Saving the standard deviation of array i
    t = 0.0
    for j in a:
        t += (pow((j-mean_i)/std_i,4)-3)
    kurtosis_i = t/len(a) # Formula: (1/N)*(summation(x_i-mean)/standard_deviation)^4-3
    return kurtosis_i


#Second difference Mean,Max,std
def sec_diff(b):
    temp1 = abs(b-b.shift(1)) # Obtaining the 1st Diffs
    t = abs(temp1-temp1.shift(1)) # Summing the 2nd Diffs
    output = t.mean() # Calculating the mean of the 2nd Diffs
    return [t.mean(),t.max(),t.std()]

#Skewness
def skewness(a):
    import scipy.stats as sp
    skew_array=sp.stats.skew(a,axis=0,bias=True)
    return skew_array #returning skewness


#First Difference Mean,Max,std
def first_diff_mean(a):
    output = abs(a-a.shift(1)) # Obtaining the 1st Diffs
    return [output.mean(),output.max(),output.std()] #returns first diff mean,max,min

#First Difference
def first_diff(a):
    if str(type(a))!="<class 'pandas.core.series.Series'>":
        a=pd.DataFrame(a)[0]
    output = a-a.shift(1)# Obtaining the 1st Diffs
    return output


#wavelet features
def wavelet_features(epoch): 
    wfeatures = []
    
    #calculating the coefficients of wavelet transform.
    cA_values,cD_values=pywt.dwt(epoch,'coif1')
    
    cA_square=np.array([x for x in (np.square(cA_values)).tolist() if x != 0])
    cD_square=np.array([x for x in (np.square(cD_values)).tolist() if x != 0])
    
      
    wfeatures.append(np.mean(cA_values)) #cA_mean
    wfeatures.append(abs(np.std(cA_values))) #cA_std
    wfeatures.append(abs(np.sum(np.square(cA_values)))) #cA_Energy
    wfeatures.append(abs(np.sum(cA_square * np.log(cA_square)))) #Entropy_A
    
    wfeatures.append(np.mean(cD_values)) #cD_mean
    wfeatures.append(abs(np.std(cD_values))) #cD_std
    wfeatures.append(abs(np.sum(np.square(cD_values)))) #cD_Energy
    wfeatures.append(abs(np.sum(cD_square * np.log(cD_square)))) #Entropy_D

    return wfeatures # returning 'Wavelet Approximate Mean','Wavelet Approximate Std Deviation','Wavelet Approximate Energy','Wavelet Approximate Entropy','Wavelet Detailed Mean','Wavelet Detailed Std Deviation','Wavelet Detailed Energy','Wavelet Detailed Entropy'


In [8]:
def features_names():
    # Generating time feature names
    
    # time domain magnitude signals' names
    magnitude_signals=['t_body_acc_X','t_body_acc_Y','t_body_acc_Z',
                       't_grav_acc_X','t_grav_acc_Y','t_grav_acc_Z',
                       't_body_acc_jerk_X','t_body_acc_jerk_Y','t_body_acc_jerk_Z',
                       't_body_acc_Mag','t_grav_acc_Mag','t_body_acc_jerk_Mag']

    # functions' names:
    t_one_input_features_name1=['_katz()','_coeff_var()']

    t_one_input_features_slope=['_slope_mean()','_slope_std()']

    t_one_input_features_hjorth=['_hjorth_activity()','_hjorth_mobility()','_hjorth_complexity()']

    t_one_input_features_sec_diff=['_sec_diff_mean()','_sec_diff_max()','_sec_diff_std()']
    t_one_input_features_first_diff=['_first_diff_mean()','_first_diff_max()','_first_diff_std()']
    
    t_one_input_features_wavelet=['_wavelet_cA_mean()','_wavelet_cA_std()','_wavelet_cA_Energy()','_wavelet_Entropy_A()',
                                  '_wavelet_cD_mean()','_wavelet_cD_std()','_wavelet_cD_Energy()','_wavelet_Entropy_D()']
    
    features=[]# Empty list : it will contain all time domain features' names
    
    for columns in magnitude_signals: # iterate throw time domain magnitude column names

        # build feature names related to that column
        #list 1
        for feature in t_one_input_features_name1:
            newcolumn=columns+feature
            features.append(newcolumn)
        
        # list 2
        for feature in t_one_input_features_slope: 
            newcolumn=columns+feature
            features.append(newcolumn)
            
       
        # list 3
        for feature in t_one_input_features_hjorth:
            newcolumn=columns+feature
            features.append(newcolumn)
            
        # list 4
        for feature in t_one_input_features_sec_diff:
            newcolumn=columns+feature
            features.append(newcolumn)
        # list 5
        for feature in t_one_input_features_first_diff:
            newcolumn=columns+feature
            features.append(newcolumn)
        # list 6
        for feature in t_one_input_features_wavelet:
            newcolumn=columns+feature
            features.append(newcolumn)
            
        
    ###########################################################################################################
    time_list_features=features
    
    return time_list_features # return all time domain features' names

In [9]:
def features_generation(t_window):
    
    # select mag columns : the last 3 columns in a time domain window
    
    mag_columns=t_window.columns # mag columns' names
    
    t_mag_features=[] # a global list will contain all time domain magnitude features
    
    for col in mag_columns: # iterate throw each mag column
        
        fkatz = [katz(t_window[col])] # 1 value
        fcoeff_var   = [coeff_var(t_window[col])] # 1 value
        fslope    = slope(t_window[col])# 2 value
        fhjorth    = hjorth(t_window[col])# 3 value
        fsec_diff    = sec_diff(t_window[col])# 3 value
        ffirst_diff_mean = first_diff_mean(t_window[col])# 3 value
        fwavelet_features    = wavelet_features(t_window[col])# 8 value
        
        # 13 value per each t_mag_column
        col_mag_values = fkatz+fcoeff_var+fslope+fhjorth+fsec_diff+ffirst_diff_mean+fwavelet_features
        
        # col_mag_values will be added to the global list
        t_mag_features= t_mag_features+ col_mag_values
    
    # t_mag_features contains 65 values = 13 values (per each t_mag_column) x 5 (t_mag_columns)
    return t_mag_features
 

def Dataset_Generation_PipeLine(b):
    data,sampling_freq=preprocess_real_smartphone(pd.read_csv(b))
    time_sig_df=time_domain_signal(data,sampling_freq)
    # concatenate all features and append the activity id and the user id
    row= features_generation(time_sig_df)
    return(row)




In [10]:
from joblib import Parallel, delayed
import multiprocessing
import time

# Smartphone

### training

In [13]:
a_real=glob.glob("training_data/smartphone_accelerometer/*.csv")
start_time = time.time()
num_cores = multiprocessing.cpu_count()
result=Parallel(n_jobs=num_cores)(delayed(Dataset_Generation_PipeLine)(i) for i in a_real)
print("--- %s Mins ---" % ((time.time() - start_time)/60))
#20.544 min 

--- 20.54461863040924 Mins ---


In [14]:
training_data=pd.DataFrame(result)
training_data.columns=features_names()
training_data['measurement_id']=[item[len('training_data/smartphone_accelerometer/'):-4] for item in a_real]
print(training_data.shape)
training_data.head()

(526, 253)


,t_body_acc_X_katz(),t_body_acc_X_coeff_var(),t_body_acc_X_slope_mean(),t_body_acc_X_slope_std(),t_body_acc_X_hjorth_activity(),t_body_acc_X_hjorth_mobility(),t_body_acc_X_hjorth_complexity(),t_body_acc_X_sec_diff_mean(),t_body_acc_X_sec_diff_max(),t_body_acc_X_sec_diff_std(),...,t_body_acc_jerk_Mag_first_diff_std(),t_body_acc_jerk_Mag_wavelet_cA_mean(),t_body_acc_jerk_Mag_wavelet_cA_std(),t_body_acc_jerk_Mag_wavelet_cA_Energy(),t_body_acc_jerk_Mag_wavelet_Entropy_A(),t_body_acc_jerk_Mag_wavelet_cD_mean(),t_body_acc_jerk_Mag_wavelet_cD_std(),t_body_acc_jerk_Mag_wavelet_cD_Energy(),t_body_acc_jerk_Mag_wavelet_Entropy_D(),measurement_id
0,1.000046,-474035.467275,-0.000013,0.026462,0.037756,0.182010,4.080593,0.004415,1.830736,0.020860,...,29.383219,4.103529,48.020357,1.393699e+08,2.186716e+09,-0.000474,22.197886,2.956526e+07,4.334669e+08,05205dc3-9c82-4b6b-921e-b6c2edd2c993
1,1.003821,202140.199789,-0.000983,0.328246,1.061892,0.340858,2.032150,0.096267,4.764164,0.179184,...,22.172472,47.003156,65.877710,3.929567e+08,4.251168e+09,-0.001581,16.295724,1.593330e+07,1.424324e+08,aba31c29-79ef-4221-9412-156538a2fd4e
2,1.000324,-64395.594048,0.000024,0.083277,0.142422,0.281528,2.455005,0.014698,4.927320,0.059402,...,10.461080,11.883184,30.390764,6.388958e+07,7.231263e+08,-0.001341,7.486253,3.362695e+06,3.097115e+07,6cab466b-3c95-4cc0-ba1b-8650f5a12504
3,1.000060,-58766.906591,0.000060,0.030454,0.013962,0.321067,1.839911,0.010312,0.726940,0.016041,...,2.639720,5.301139,7.415292,4.985401e+06,3.436853e+07,-0.001355,1.945919,2.272000e+05,1.203258e+06,27eccfc4-e329-4695-aee8-6d706b247191
4,1.002672,4575.288448,0.002206,0.464255,0.285314,0.662930,2.166208,0.079449,24.713101,0.293832,...,16.007185,13.712178,33.357373,2.690837e+07,2.934527e+08,0.046870,11.206863,2.598204e+06,2.343991e+07,ed560c25-e5c5-4dba-82c7-3fc18c248ce4


In [15]:
#ancillary_data
b_real=glob.glob("ancillary_data/smartphone_accelerometer/*.csv")
b_real2=[]
for i in b_real:
    a=pd.read_csv(i).shape[0]
    if a>=2000:
        b_real2.append(i)


start_time = time.time()
num_cores = multiprocessing.cpu_count()
result=Parallel(n_jobs=num_cores)(delayed(Dataset_Generation_PipeLine)(i) for i in b_real2)
print("--- %s Mins ---" % ((time.time() - start_time)/60))
#11.363 min 

/Users/akuma824/Desktop/anaconda3/lib/python3.7/site-packages/joblib/externals/loky/process_executor.py:706: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  "timeout or by a memory leak.", UserWarning


--- 11.363233252366383 Mins ---


In [16]:
df_ancillary=pd.DataFrame(result)
df_ancillary.columns=features_names()
df_ancillary['measurement_id']=[item[len('ancillary_data/smartphone_accelerometer/'):-4] for item in b_real2]
print(df_ancillary.shape)
df_ancillary.head()

(348, 253)


,t_body_acc_X_katz(),t_body_acc_X_coeff_var(),t_body_acc_X_slope_mean(),t_body_acc_X_slope_std(),t_body_acc_X_hjorth_activity(),t_body_acc_X_hjorth_mobility(),t_body_acc_X_hjorth_complexity(),t_body_acc_X_sec_diff_mean(),t_body_acc_X_sec_diff_max(),t_body_acc_X_sec_diff_std(),...,t_body_acc_jerk_Mag_first_diff_std(),t_body_acc_jerk_Mag_wavelet_cA_mean(),t_body_acc_jerk_Mag_wavelet_cA_std(),t_body_acc_jerk_Mag_wavelet_cA_Energy(),t_body_acc_jerk_Mag_wavelet_Entropy_A(),t_body_acc_jerk_Mag_wavelet_cD_mean(),t_body_acc_jerk_Mag_wavelet_cD_std(),t_body_acc_jerk_Mag_wavelet_cD_Energy(),t_body_acc_jerk_Mag_wavelet_Entropy_D(),measurement_id
0,1.000000,-587966.420697,-7.193507e-07,0.000887,0.000058,0.132900,1.403225,0.000143,0.001494,0.000111,...,0.060253,1.199134,0.515406,5.110732e+05,4.319079e+05,0.000002,0.025210,1.906641e+02,1.127702e+03,eab1cc17-40aa-4e1f-8e4f-64e6d5743509
1,1.000315,48894.202072,4.516919e-04,0.103361,0.185877,0.242232,3.154527,0.016163,7.566209,0.061289,...,7.190025,9.733553,21.477499,3.336206e+07,3.274044e+08,-0.004947,5.046609,1.528123e+06,1.333829e+07,b1a5fd6d-db9c-4870-a3c0-943e0656d112
2,1.003249,-5759.390980,2.671911e-03,0.530950,0.402334,0.537203,2.177747,0.071791,13.667867,0.270522,...,11.462667,10.435900,26.593493,1.566791e+07,1.543660e+08,0.021199,7.896549,1.197109e+06,9.339204e+06,bca5e12d-9fd6-496b-ac08-9e2472d8b299
3,1.000655,-36516.433629,-4.700092e-04,0.107656,0.171133,0.312366,2.116920,0.032236,1.688237,0.063908,...,12.296451,21.899163,37.612554,1.136604e+08,1.099945e+09,-0.001263,8.930533,4.785424e+06,3.369217e+07,9d74f5e1-241a-4f4a-bc6f-2779edf410cd
4,1.000004,140114.724393,-1.428941e-05,0.008020,0.000118,0.823587,1.460451,0.004942,0.158282,0.005557,...,0.513419,1.153107,0.735740,7.484247e+04,1.159690e+05,-0.002240,0.465897,8.683034e+03,3.455195e+03,49f80736-6b50-44a6-a77b-9b1572334a8c


In [17]:
Frame = training_data.append(pd.DataFrame(data = df_ancillary), ignore_index=True)
print(Frame.shape)
Frame.head()

(874, 253)


,t_body_acc_X_katz(),t_body_acc_X_coeff_var(),t_body_acc_X_slope_mean(),t_body_acc_X_slope_std(),t_body_acc_X_hjorth_activity(),t_body_acc_X_hjorth_mobility(),t_body_acc_X_hjorth_complexity(),t_body_acc_X_sec_diff_mean(),t_body_acc_X_sec_diff_max(),t_body_acc_X_sec_diff_std(),...,t_body_acc_jerk_Mag_first_diff_std(),t_body_acc_jerk_Mag_wavelet_cA_mean(),t_body_acc_jerk_Mag_wavelet_cA_std(),t_body_acc_jerk_Mag_wavelet_cA_Energy(),t_body_acc_jerk_Mag_wavelet_Entropy_A(),t_body_acc_jerk_Mag_wavelet_cD_mean(),t_body_acc_jerk_Mag_wavelet_cD_std(),t_body_acc_jerk_Mag_wavelet_cD_Energy(),t_body_acc_jerk_Mag_wavelet_Entropy_D(),measurement_id
0,1.000046,-474035.467275,-0.000013,0.026462,0.037756,0.182010,4.080593,0.004415,1.830736,0.020860,...,29.383219,4.103529,48.020357,1.393699e+08,2.186716e+09,-0.000474,22.197886,2.956526e+07,4.334669e+08,05205dc3-9c82-4b6b-921e-b6c2edd2c993
1,1.003821,202140.199789,-0.000983,0.328246,1.061892,0.340858,2.032150,0.096267,4.764164,0.179184,...,22.172472,47.003156,65.877710,3.929567e+08,4.251168e+09,-0.001581,16.295724,1.593330e+07,1.424324e+08,aba31c29-79ef-4221-9412-156538a2fd4e
2,1.000324,-64395.594048,0.000024,0.083277,0.142422,0.281528,2.455005,0.014698,4.927320,0.059402,...,10.461080,11.883184,30.390764,6.388958e+07,7.231263e+08,-0.001341,7.486253,3.362695e+06,3.097115e+07,6cab466b-3c95-4cc0-ba1b-8650f5a12504
3,1.000060,-58766.906591,0.000060,0.030454,0.013962,0.321067,1.839911,0.010312,0.726940,0.016041,...,2.639720,5.301139,7.415292,4.985401e+06,3.436853e+07,-0.001355,1.945919,2.272000e+05,1.203258e+06,27eccfc4-e329-4695-aee8-6d706b247191
4,1.002672,4575.288448,0.002206,0.464255,0.285314,0.662930,2.166208,0.079449,24.713101,0.293832,...,16.007185,13.712178,33.357373,2.690837e+07,2.934527e+08,0.046870,11.206863,2.598204e+06,2.343991e+07,ed560c25-e5c5-4dba-82c7-3fc18c248ce4


In [19]:
Frame.to_csv('realpd_comp_training_abhiroop_lastfeatures_smartphone.csv',index=False)

### testing

In [18]:
a_real=glob.glob("testing_data/smartphone_accelerometer/*.csv")
start_time = time.time()
num_cores = multiprocessing.cpu_count()
result=Parallel(n_jobs=num_cores)(delayed(Dataset_Generation_PipeLine)(i) for i in a_real)

training_data=pd.DataFrame(result)
training_data.columns=features_names()
training_data['measurement_id']=[item[len('testing_data/smartphone_accelerometer/'):-4] for item in a_real]

Frame_test = training_data.copy()
print(Frame_test.shape)
Frame_test.head()



(169, 253)


,t_body_acc_X_katz(),t_body_acc_X_coeff_var(),t_body_acc_X_slope_mean(),t_body_acc_X_slope_std(),t_body_acc_X_hjorth_activity(),t_body_acc_X_hjorth_mobility(),t_body_acc_X_hjorth_complexity(),t_body_acc_X_sec_diff_mean(),t_body_acc_X_sec_diff_max(),t_body_acc_X_sec_diff_std(),...,t_body_acc_jerk_Mag_first_diff_std(),t_body_acc_jerk_Mag_wavelet_cA_mean(),t_body_acc_jerk_Mag_wavelet_cA_std(),t_body_acc_jerk_Mag_wavelet_cA_Energy(),t_body_acc_jerk_Mag_wavelet_Entropy_A(),t_body_acc_jerk_Mag_wavelet_cD_mean(),t_body_acc_jerk_Mag_wavelet_cD_std(),t_body_acc_jerk_Mag_wavelet_cD_Energy(),t_body_acc_jerk_Mag_wavelet_Entropy_D(),measurement_id
0,1.007186,1.259028e+06,0.004715,0.500324,2.343522,0.402286,1.602876,0.113370,13.101362,0.318258,...,32.066660,49.346485,98.683105,7.304180e+08,8.851149e+09,-0.044703,21.235846,2.705824e+07,2.607164e+08,7860035d-c9df-44e9-ba0e-0855dfad90f4
1,1.011013,2.234996e+05,-0.003220,0.524701,9.778053,0.227365,2.470623,0.134461,8.508648,0.318067,...,45.986081,86.334489,167.132938,2.123259e+09,2.702989e+10,0.063188,31.691755,6.026328e+07,5.960335e+08,8abf7688-c6bb-488f-bb9d-c359a5f86b35
2,1.000591,2.203282e+04,0.000038,0.112981,0.196748,0.298925,2.207169,0.027153,3.005660,0.068131,...,14.352802,15.243349,37.291417,9.738219e+07,1.129841e+09,-0.001102,9.159239,5.033583e+06,4.976218e+07,5b76c255-d8dd-40d3-82e9-745cb943607f
3,1.000083,-1.595735e+04,-0.000047,0.049266,0.016428,0.430944,2.033974,0.005352,4.437498,0.038832,...,7.375219,2.791829,17.730166,1.932951e+07,2.339059e+08,0.006350,5.786445,2.009013e+06,2.100553e+07,c4f0d5dc-c1db-4cc5-89d1-26d7fee32299
4,1.000197,1.905505e+05,0.000028,0.066860,0.038342,0.388051,1.772939,0.015282,3.667435,0.040560,...,4.485009,7.468222,13.972190,1.506004e+07,1.340652e+08,0.000939,3.200937,6.147700e+05,4.450083e+06,6bb1ba19-25ac-4889-8e5b-8e4215784e82


In [20]:
Frame_test.to_csv('realpd_comp_testing_abhiroop_lastfeatures_smartphone.csv',index=False)

In [21]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.decomposition import PCA
import ppscore as pps
import lightgbm as lgb
from catboost import CatBoostRegressor, FeaturesData, Pool
from sklearn import preprocessing

#Test Data
cis_pd_testing_id=pd.read_csv('test_data_Id/cis-pd.CIS-PD_Test_Data_IDs.csv')
real_pd_testing_id=pd.read_csv('test_data_Id/real-pd.REAL-PD_Test_Data_IDs.csv')

#Training Data
cis_pd_training_id=pd.read_csv('data_labels/CIS-PD_Training_Data_IDs_Labels.csv')
real_pd_training_id=pd.read_csv('data_labels/REAL-PD_Training_Data_IDs_Labels.csv')

#Ancillary Data
cis_pd_ancillary_id=pd.read_csv('data_labels/CIS-PD_Ancillary_Data_IDs_Labels.csv')
real_pd_ancillary_id=pd.read_csv('data_labels/REAL-PD_Ancillary_Data_IDs_Labels.csv')

In [22]:
#smartwatch_gyroscope features
label=real_pd_training_id.append(pd.DataFrame(real_pd_ancillary_id), ignore_index=True)
df_train=pd.merge(label,Frame,on='measurement_id')
df_train['subject_id']=preprocessing.LabelEncoder().fit(df_train['subject_id']).transform(df_train['subject_id'])
print(df_train.shape)
df_train.head()

(874, 257)


,measurement_id,subject_id,on_off,dyskinesia,tremor,t_body_acc_X_katz(),t_body_acc_X_coeff_var(),t_body_acc_X_slope_mean(),t_body_acc_X_slope_std(),t_body_acc_X_hjorth_activity(),...,t_body_acc_jerk_Mag_first_diff_max(),t_body_acc_jerk_Mag_first_diff_std(),t_body_acc_jerk_Mag_wavelet_cA_mean(),t_body_acc_jerk_Mag_wavelet_cA_std(),t_body_acc_jerk_Mag_wavelet_cA_Energy(),t_body_acc_jerk_Mag_wavelet_Entropy_A(),t_body_acc_jerk_Mag_wavelet_cD_mean(),t_body_acc_jerk_Mag_wavelet_cD_std(),t_body_acc_jerk_Mag_wavelet_cD_Energy(),t_body_acc_jerk_Mag_wavelet_Entropy_D()
0,ee053d95-c155-400d-ae42-fe24834ad4a9,2,1.0,0.0,3.0,1.001259,190525.170377,-0.000161,0.179777,0.136928,...,1000.831680,12.201951,19.186292,16.089191,1.791458e+07,1.464158e+08,-0.070816,11.577819,3.830236e+06,2.975716e+07
1,ce51ee31-8553-4321-9f83-8cd3dabe2f66,2,1.0,0.0,2.0,1.003508,64305.547204,0.000172,0.410412,0.554120,...,2080.319703,54.846396,37.624867,127.513167,1.178337e+09,1.618287e+10,0.001988,38.812255,1.004251e+08,1.162400e+09
2,5c42911d-0ebd-47ba-9925-dd5ab1c0ed61,2,1.0,0.0,1.0,1.012685,-51797.351931,-0.000411,1.089542,1.926549,...,4484.661304,95.071079,58.554900,171.905829,1.522172e+09,2.188481e+10,0.135727,70.624774,2.302105e+08,2.957325e+09
3,274f5bc8-2e4f-4d7c-a546-b65b7d6bd01e,2,0.0,0.0,NaN,1.016144,114164.595367,-0.001818,1.192039,0.988393,...,3359.982425,71.654705,48.245759,104.387826,3.778496e+08,4.850456e+09,-0.329438,53.889976,8.297989e+07,9.905229e+08
4,ecbeea40-8770-455d-90a6-597e7f896e1b,2,0.0,1.0,0.0,1.027986,-30492.945031,-0.001418,1.628427,1.697255,...,5291.004320,99.530744,76.610624,138.317694,7.143528e+08,9.206557e+09,-0.314920,75.458064,1.626952e+08,1.994225e+09


In [23]:
#lightgbm
def lightgbm(df4,label):
    df4=df4[df4['tremor']!=4]
    train=df4[~df4[label].isnull()]

    # Splitting the dataset into the Training set and Test set
    x_train, x_test, y_train, y_test = train_test_split(train.drop(['subject_id','measurement_id','on_off','dyskinesia','tremor'], axis = 1), train[label], 
                                                        train_size=0.7, 
                                                        stratify = train[label],
                                                        random_state=1)

    d_train = lgb.Dataset(x_train, label=y_train)

    params = {}
    params['learning_rate'] = 0.03
    params['boosting_type'] = 'gbdt'
    params['objective'] = 'regression'
    params['metric'] = 'rmse'
    params['max_depth'] = 5
    params['feature_fraction'] = 0.8
    clf = lgb.train(params, d_train, 150)
    y_pred=clf.predict(x_test)
    y_pred2=clf.predict(x_train)

    #print('lightgbm train_MSE '+label+' :',round(mean_squared_error(y_train, y_pred2),3))
    print('lightgbm test_MSE '+label+' :',round(mean_squared_error(y_test, y_pred),3))
    
    
#Catboost    
#This will give indexes of the categorical features
def categorical_index(df,cols):
    cat=[]
    for c in cols:
        try:
            cat.append(df.columns.get_loc(c))
        except:
            pass # doing nothing on exception
    return cat

def catboost(df4,label): 
    df4=df4[df4['tremor']!=4]
    train=df4[~df4[label].isnull()]
    X_train, X_test, y_train, y_test = train_test_split(train.drop(['subject_id','measurement_id','on_off','dyskinesia','tremor'], axis = 1), train[label], 
                                                        train_size=0.7, 
                                                        stratify = train[label],
                                                        random_state=1)
    train_dataset = Pool(data=X_train,
                         label=y_train,
                         cat_features=categorical_index(X_train,[]))

    eval_dataset = Pool(data=X_test,
                        label=y_test,
                        cat_features=categorical_index(X_train,[]))



    model = CatBoostRegressor(iterations=50, learning_rate=0.1, depth=6,verbose=False)
    # Fit model
    model.fit(train_dataset)
    # Get predictions
    preds = model.predict(eval_dataset)
    preds2 = model.predict(train_dataset)

    #print('catboost train_MSE '+label+' :',round(mean_squared_error(y_train, preds2),3))
    print('catboost test_MSE '+label+' :',round(mean_squared_error(y_test, preds),3))

In [24]:
lightgbm(df_train,'tremor')
catboost(df_train,'tremor')

lightgbm(df_train,'dyskinesia')
catboost(df_train,'dyskinesia')

lightgbm(df_train,'on_off')
catboost(df_train,'on_off')

lightgbm test_MSE tremor : 0.603
catboost test_MSE tremor : 0.636
lightgbm test_MSE dyskinesia : 0.132
catboost test_MSE dyskinesia : 0.142
lightgbm test_MSE on_off : 0.201
catboost test_MSE on_off : 0.196


In [ ]:
lightgbm test_MSE tremor : 0.603
catboost test_MSE tremor : 0.636
lightgbm test_MSE dyskinesia : 0.132
catboost test_MSE dyskinesia : 0.142
lightgbm test_MSE on_off : 0.201
catboost test_MSE on_off : 0.196

# SmartWatch

## Accelerometer

In [25]:
def preprocess_real_smartwatch(data):
    
    a=data.groupby('device_id').agg({'x':'var','y':'count'}).reset_index()
    deviceid=a.loc[a.x.idxmax(),'device_id']
    if int(a.loc[a.device_id==deviceid,'y'])<=data.shape[0]*0.2:
        deviceid=a.loc[a.x.idxmin(),'device_id']
    
    data=data[data.device_id==deviceid].reset_index()
    data.rename(columns={'t':'Timestamp','x':'X','y':'Y','z':'Z'},inplace=True)
   
    ls=['X','Y','Z']
    #freq=round((1/((data.Timestamp.max()/data.Timestamp.shape[0]).round(3))),0)
    t1=np.arange(data.Timestamp[0],data.Timestamp[(data.shape[0])-1],0.02)
    freq=50
    #t1=np.arange(data.Timestamp[0],data.Timestamp[(data.shape[0])-1],(data.Timestamp.max()/data.Timestamp.shape[0]).round(3))
    df=pd.DataFrame({'Timestamp':t1})
        
    for i in ls:
        fcubic = interpolate.interp1d(data.Timestamp, data[i])
        df[i]=fcubic(t1)
    df.rename(columns={'X':'acc_X','Y':'acc_Y','Z':'acc_Z'},inplace=True)
    return df[['Timestamp','acc_X','acc_Y','acc_Z']],deviceid,freq

def Dataset_Generation_PipeLine_smartwatch_acc(b):
    data,device_id,sampling_freq=preprocess_real_smartwatch(pd.read_csv(b))
    time_sig_df=time_domain_signal(data,sampling_freq)
    # concatenate all features and append the activity id and the user id
    row= features_generation(time_sig_df)
    return([device_id]+row)


In [26]:
a_real=glob.glob("training_data/smartwatch_accelerometer/*.csv")
a_real_acc=[]
num=[]
for i in a_real:
    data=pd.read_csv(i)
    a=data.groupby('device_id').agg({'x':'var','y':'count'}).reset_index()
    deviceid=a.loc[a.x.idxmax(),'device_id']
    if int(a.loc[a.device_id==deviceid,'y'])<=data.shape[0]*0.2:
        deviceid=a.loc[a.x.idxmin(),'device_id']
    data=data[data.device_id==deviceid].reset_index(drop=True)
    num.append(data.shape[0])
    if data.shape[0]>=2000:
        a_real_acc.append(i)

In [27]:
len(a_real_acc),len(a_real)

(530, 535)

In [28]:
import time
start_time = time.time()
num_cores = multiprocessing.cpu_count()
result=Parallel(n_jobs=num_cores)(delayed(Dataset_Generation_PipeLine_smartwatch_acc)(i) for i in a_real_acc)
print("--- %s Mins ---" % ((time.time() - start_time)/60))
#12.101 min 

--- 12.63281877040863 Mins ---


In [29]:
training_data=pd.DataFrame(result)
training_data.columns=['device_id_acc']+features_names()
training_data['measurement_id']=[item[len('training_data/smartphone_accelerometer/'):-4] for item in a_real_acc]
print(training_data.shape)
training_data.head()

(530, 254)


,device_id_acc,t_body_acc_X_katz(),t_body_acc_X_coeff_var(),t_body_acc_X_slope_mean(),t_body_acc_X_slope_std(),t_body_acc_X_hjorth_activity(),t_body_acc_X_hjorth_mobility(),t_body_acc_X_hjorth_complexity(),t_body_acc_X_sec_diff_mean(),t_body_acc_X_sec_diff_max(),...,t_body_acc_jerk_Mag_first_diff_std(),t_body_acc_jerk_Mag_wavelet_cA_mean(),t_body_acc_jerk_Mag_wavelet_cA_std(),t_body_acc_jerk_Mag_wavelet_cA_Energy(),t_body_acc_jerk_Mag_wavelet_Entropy_A(),t_body_acc_jerk_Mag_wavelet_cD_mean(),t_body_acc_jerk_Mag_wavelet_cD_std(),t_body_acc_jerk_Mag_wavelet_cD_Energy(),t_body_acc_jerk_Mag_wavelet_Entropy_D(),measurement_id
0,2VSP,1.004000,-24121.668930,-0.000043,0.274105,1.195276,0.322246,3.282729,0.122494,7.748725,...,24.246612,33.134610,52.775730,1.164993e+08,1.218558e+09,-0.128587,18.076180,9.803271e+06,8.351328e+07,05205dc3-9c82-4b6b-921e-b6c2edd2c993
1,2WLT,1.020188,58603.225645,0.000753,0.942833,5.940303,0.402234,2.863228,0.413388,24.422981,...,52.917483,101.232822,108.388156,6.599043e+08,7.820029e+09,0.337225,42.453446,5.407407e+07,5.443075e+08,aba31c29-79ef-4221-9412-156538a2fd4e
2,2ZX3,1.002798,-545517.997583,0.001156,0.228635,0.662064,0.373940,2.764917,0.088138,5.926416,...,15.393561,19.739822,36.443500,5.153540e+07,5.113802e+08,0.052770,11.645678,4.068874e+06,3.254278e+07,6cab466b-3c95-4cc0-ba1b-8650f5a12504
3,2WH7,1.000423,-144341.044526,0.000079,0.088970,0.095432,0.330273,4.100591,0.046259,3.171407,...,6.341025,9.828324,12.725178,7.756042e+06,6.189171e+07,-0.010777,5.004624,7.514164e+05,4.854295e+06,ed560c25-e5c5-4dba-82c7-3fc18c248ce4
4,2WH7,1.001493,39152.736486,0.000002,0.198279,0.312401,0.415120,3.209567,0.070423,6.303599,...,15.475185,16.717025,31.942369,3.899581e+07,3.959978e+08,-0.046825,11.916498,4.260438e+06,3.749578e+07,f96752b5-850e-4a5a-a74a-69ab4893b6aa


In [30]:
a_real=glob.glob("ancillary_data/smartwatch_accelerometer/*.csv")
a_real_acc=[]
num=[]
for i in a_real:
    data=pd.read_csv(i)
    a=data.groupby('device_id').agg({'x':'var','y':'count'}).reset_index()
    deviceid=a.loc[a.x.idxmax(),'device_id']
    if int(a.loc[a.device_id==deviceid,'y'])<=data.shape[0]*0.2:
        deviceid=a.loc[a.x.idxmin(),'device_id']
    data=data[data.device_id==deviceid].reset_index(drop=True)
    num.append(data.shape[0])
    if data.shape[0]>=2000:
        a_real_acc.append(i)

In [31]:
len(a_real_acc),len(a_real)

(426, 428)

In [32]:
import time
start_time = time.time()
num_cores = multiprocessing.cpu_count()
result=Parallel(n_jobs=num_cores)(delayed(Dataset_Generation_PipeLine_smartwatch_acc)(i) for i in a_real_acc)
print("--- %s Mins ---" % ((time.time() - start_time)/60))
#10.233 min 

--- 10.093303966522218 Mins ---


In [33]:
df_ancillary=pd.DataFrame(result)
df_ancillary.columns=['device_id_acc']+features_names()
df_ancillary['measurement_id']=[item[len('ancillary_data/smartwatch_accelerometer/'):-4] for item in a_real_acc]
print(df_ancillary.shape)
df_ancillary.head()

(426, 254)


,device_id_acc,t_body_acc_X_katz(),t_body_acc_X_coeff_var(),t_body_acc_X_slope_mean(),t_body_acc_X_slope_std(),t_body_acc_X_hjorth_activity(),t_body_acc_X_hjorth_mobility(),t_body_acc_X_hjorth_complexity(),t_body_acc_X_sec_diff_mean(),t_body_acc_X_sec_diff_max(),...,t_body_acc_jerk_Mag_first_diff_std(),t_body_acc_jerk_Mag_wavelet_cA_mean(),t_body_acc_jerk_Mag_wavelet_cA_std(),t_body_acc_jerk_Mag_wavelet_cA_Energy(),t_body_acc_jerk_Mag_wavelet_Entropy_A(),t_body_acc_jerk_Mag_wavelet_cD_mean(),t_body_acc_jerk_Mag_wavelet_cD_std(),t_body_acc_jerk_Mag_wavelet_cD_Energy(),t_body_acc_jerk_Mag_wavelet_Entropy_D(),measurement_id
0,2WLT,1.002863,9048.142586,0.000065,0.291877,0.241027,0.622906,2.416673,0.102471,7.293960,...,17.302626,23.742797,38.617638,6.165538e+07,6.059524e+08,-0.074499,13.231625,5.252794e+06,4.076471e+07,eab1cc17-40aa-4e1f-8e4f-64e6d5743509
1,2WH7,1.000218,8365.349311,-0.000165,0.062554,0.013268,0.620810,2.071451,0.034911,1.926634,...,3.734114,6.488661,7.611613,2.771591e+06,1.915645e+07,-0.003879,3.048874,2.575360e+05,1.349995e+06,166ba983-209f-4639-a5a6-d6e66adeba2b
2,2WH5,1.002679,85215.011311,0.000217,0.276775,0.485745,0.473479,2.669775,0.084079,12.410336,...,19.363182,18.086702,44.079662,5.749597e+07,6.236504e+08,0.037479,14.065864,5.010945e+06,4.434523e+07,bca5e12d-9fd6-496b-ac08-9e2472d8b299
3,327T,1.000114,72642.747488,-0.000149,0.048892,0.004270,0.767195,1.906556,0.032456,0.229525,...,1.635120,5.893948,2.136874,1.179185e+06,4.613253e+06,-0.014784,1.838239,1.013836e+05,2.079656e+05,49f80736-6b50-44a6-a77b-9b1572334a8c
4,327T,1.000442,-12556.426237,0.000123,0.085459,0.097952,0.333752,3.496981,0.045014,3.616567,...,4.708086,8.936002,10.397893,5.639237e+06,4.077461e+07,0.026040,3.858474,4.466698e+05,2.331490e+06,26f49660-ce1a-4946-8f83-f88850f03ec1


In [34]:
Frame_smartwatch_acc = training_data.append(pd.DataFrame(data = df_ancillary), ignore_index=True)
print(Frame_smartwatch_acc.shape)
Frame_smartwatch_acc.head()

(956, 254)


,device_id_acc,t_body_acc_X_katz(),t_body_acc_X_coeff_var(),t_body_acc_X_slope_mean(),t_body_acc_X_slope_std(),t_body_acc_X_hjorth_activity(),t_body_acc_X_hjorth_mobility(),t_body_acc_X_hjorth_complexity(),t_body_acc_X_sec_diff_mean(),t_body_acc_X_sec_diff_max(),...,t_body_acc_jerk_Mag_first_diff_std(),t_body_acc_jerk_Mag_wavelet_cA_mean(),t_body_acc_jerk_Mag_wavelet_cA_std(),t_body_acc_jerk_Mag_wavelet_cA_Energy(),t_body_acc_jerk_Mag_wavelet_Entropy_A(),t_body_acc_jerk_Mag_wavelet_cD_mean(),t_body_acc_jerk_Mag_wavelet_cD_std(),t_body_acc_jerk_Mag_wavelet_cD_Energy(),t_body_acc_jerk_Mag_wavelet_Entropy_D(),measurement_id
0,2VSP,1.004000,-24121.668930,-0.000043,0.274105,1.195276,0.322246,3.282729,0.122494,7.748725,...,24.246612,33.134610,52.775730,1.164993e+08,1.218558e+09,-0.128587,18.076180,9.803271e+06,8.351328e+07,05205dc3-9c82-4b6b-921e-b6c2edd2c993
1,2WLT,1.020188,58603.225645,0.000753,0.942833,5.940303,0.402234,2.863228,0.413388,24.422981,...,52.917483,101.232822,108.388156,6.599043e+08,7.820029e+09,0.337225,42.453446,5.407407e+07,5.443075e+08,aba31c29-79ef-4221-9412-156538a2fd4e
2,2ZX3,1.002798,-545517.997583,0.001156,0.228635,0.662064,0.373940,2.764917,0.088138,5.926416,...,15.393561,19.739822,36.443500,5.153540e+07,5.113802e+08,0.052770,11.645678,4.068874e+06,3.254278e+07,6cab466b-3c95-4cc0-ba1b-8650f5a12504
3,2WH7,1.000423,-144341.044526,0.000079,0.088970,0.095432,0.330273,4.100591,0.046259,3.171407,...,6.341025,9.828324,12.725178,7.756042e+06,6.189171e+07,-0.010777,5.004624,7.514164e+05,4.854295e+06,ed560c25-e5c5-4dba-82c7-3fc18c248ce4
4,2WH7,1.001493,39152.736486,0.000002,0.198279,0.312401,0.415120,3.209567,0.070423,6.303599,...,15.475185,16.717025,31.942369,3.899581e+07,3.959978e+08,-0.046825,11.916498,4.260438e+06,3.749578e+07,f96752b5-850e-4a5a-a74a-69ab4893b6aa


### test

In [35]:
a_real=glob.glob("testing_data/smartwatch_accelerometer/*.csv")
a_real_acc=[]
num=[]
for i in a_real:
    data=pd.read_csv(i)
    a=data.groupby('device_id').agg({'x':'var','y':'count'}).reset_index()
    deviceid=a.loc[a.x.idxmax(),'device_id']
    if int(a.loc[a.device_id==deviceid,'y'])<=data.shape[0]*0.2:
        deviceid=a.loc[a.x.idxmin(),'device_id']
    data=data[data.device_id==deviceid].reset_index(drop=True)
    num.append(data.shape[0])
    if data.shape[0]>=2000:
        a_real_acc.append(i)

print(len(a_real_acc),len(a_real))

import time
start_time = time.time()
num_cores = multiprocessing.cpu_count()
result=Parallel(n_jobs=num_cores)(delayed(Dataset_Generation_PipeLine_smartwatch_acc)(i) for i in a_real_acc)

training_data=pd.DataFrame(result)
training_data.columns=['device_id_acc']+features_names()
training_data['measurement_id']=[item[len('testing_data/smartphone_accelerometer/'):-4] for item in a_real_acc]

Frame_smartwatch_acc_test = training_data.copy()
print(Frame_smartwatch_acc_test.shape)
Frame_smartwatch_acc_test.head()

171 172
(171, 254)


,device_id_acc,t_body_acc_X_katz(),t_body_acc_X_coeff_var(),t_body_acc_X_slope_mean(),t_body_acc_X_slope_std(),t_body_acc_X_hjorth_activity(),t_body_acc_X_hjorth_mobility(),t_body_acc_X_hjorth_complexity(),t_body_acc_X_sec_diff_mean(),t_body_acc_X_sec_diff_max(),...,t_body_acc_jerk_Mag_first_diff_std(),t_body_acc_jerk_Mag_wavelet_cA_mean(),t_body_acc_jerk_Mag_wavelet_cA_std(),t_body_acc_jerk_Mag_wavelet_cA_Energy(),t_body_acc_jerk_Mag_wavelet_Entropy_A(),t_body_acc_jerk_Mag_wavelet_cD_mean(),t_body_acc_jerk_Mag_wavelet_cD_std(),t_body_acc_jerk_Mag_wavelet_cD_Energy(),t_body_acc_jerk_Mag_wavelet_Entropy_D(),measurement_id
0,2ZX6,1.023602,88972.650089,0.002406,1.065472,6.124654,0.471469,2.507854,0.465609,55.776289,...,65.279374,113.248347,120.618340,8.212465e+08,9.766180e+09,-0.169012,54.844725,9.024218e+07,9.845052e+08,7860035d-c9df-44e9-ba0e-0855dfad90f4
1,2WH8,1.002468,-9577.625612,0.000620,0.251940,0.497761,0.419816,2.870803,0.092780,14.599304,...,18.905019,20.222352,40.507956,6.149719e+07,6.818394e+08,-0.175286,14.146314,6.004668e+06,5.915454e+07,0c593c3f-6636-4f0f-b9b1-f489c1cd3852
2,2VSP,1.004413,-975708.091118,-0.000236,0.360364,0.575081,0.488049,3.127990,0.149894,7.094872,...,20.596159,31.794656,43.792402,8.786610e+07,8.898007e+08,0.122876,16.042862,7.722171e+06,6.206024e+07,8abf7688-c6bb-488f-bb9d-c359a5f86b35
3,2VSP,1.000875,67926.655283,-0.000399,0.143272,0.296430,0.294123,4.185960,0.059950,8.542191,...,9.122390,13.973823,20.252368,1.816400e+07,1.612360e+08,-0.088866,6.878839,1.419885e+06,9.548193e+06,c4f0d5dc-c1db-4cc5-89d1-26d7fee32299
4,2WLT,1.002326,-65635.078837,0.000161,0.274812,0.545020,0.395441,3.183180,0.100699,13.317348,...,27.905948,27.226059,48.889858,9.395056e+07,1.090616e+09,0.068849,20.474143,1.257670e+07,1.406613e+08,6bb1ba19-25ac-4889-8e5b-8e4215784e82


## Gyroscope

In [36]:
def preprocess_real_gyroscope(data):
    
    a=data.groupby('device_id').agg({'x':'var','y':'count'}).reset_index()
    deviceid=a.loc[a.x.idxmax(),'device_id']
    if int(a.loc[a.device_id==deviceid,'y'])<=data.shape[0]*0.2:
        deviceid=a.loc[a.x.idxmin(),'device_id']
    
    data=data[data.device_id==deviceid].reset_index()
    data.rename(columns={'t':'Timestamp','x':'X','y':'Y','z':'Z'},inplace=True)
   
    ls=['X','Y','Z']
    #freq=round((1/((data.Timestamp.max()/data.Timestamp.shape[0]).round(3))),0)
    t1=np.arange(data.Timestamp[0],data.Timestamp[(data.shape[0])-1],0.02)
    freq=50
    #t1=np.arange(data.Timestamp[0],data.Timestamp[(data.shape[0])-1],(data.Timestamp.max()/data.Timestamp.shape[0]).round(3))
    df=pd.DataFrame({'Timestamp':t1})
        
    for i in ls:
        fcubic = interpolate.interp1d(data.Timestamp, data[i])
        df[i]=fcubic(t1)
    df.rename(columns={'X':'acc_X','Y':'acc_Y','Z':'acc_Z'},inplace=True)
    return df[['Timestamp','acc_X','acc_Y','acc_Z']],deviceid,freq

def Dataset_Generation_PipeLine_smartwatch_gyro(b):
    data,device_id,sampling_freq=preprocess_real_gyroscope(pd.read_csv(b))
    time_sig_df=time_domain_signal(data,sampling_freq)
    time_sig_df=time_sig_df.drop(['t_grav_acc_X','t_grav_acc_Y','t_grav_acc_Z','t_grav_acc_mag'],axis=1)
    # concatenate all features and append the activity id and the user id
    row= features_generation(time_sig_df)
    return([device_id]+row)

In [37]:
fn_gyro=[i.replace('acc','gyro') for i in features_names()]
fn_gyro=[i for i in fn_gyro if not any(w in 'grav' for w in i.split('_'))]

In [38]:
a_real=glob.glob("training_data/smartwatch_gyroscope/*.csv")
a_real_gyro=[]
num=[]
for i in a_real:
    data=pd.read_csv(i)
    a=data.groupby('device_id').agg({'x':'var','y':'count'}).reset_index()
    deviceid=a.loc[a.x.idxmax(),'device_id']
    if int(a.loc[a.device_id==deviceid,'y'])<=data.shape[0]*0.2:
        deviceid=a.loc[a.x.idxmin(),'device_id']
    data=data[data.device_id==deviceid].reset_index(drop=True)
    num.append(data.shape[0])
    if data.shape[0]>=2000:
        a_real_gyro.append(i)

In [39]:
len(a_real_gyro),len(a_real)

(530, 535)

In [40]:
# smartwatch_accelerometer
start_time = time.time()
num_cores = multiprocessing.cpu_count()
result=Parallel(n_jobs=num_cores)(delayed(Dataset_Generation_PipeLine_smartwatch_gyro)(i) for i in a_real_gyro)
print("--- %s Mins ---" % ((time.time() - start_time)/60))
#11.900 Mins

--- 11.90065267086029 Mins ---


In [41]:
training_data=pd.DataFrame(result)
training_data.columns=['device_id_gyro']+fn_gyro
training_data['measurement_id']=[item[len('training_data/smartwatch_gyroscope/'):-4] for item in a_real_gyro]
print(training_data.shape)
training_data.head()

(530, 170)


,device_id_gyro,t_body_gyro_X_katz(),t_body_gyro_X_coeff_var(),t_body_gyro_X_slope_mean(),t_body_gyro_X_slope_std(),t_body_gyro_X_hjorth_activity(),t_body_gyro_X_hjorth_mobility(),t_body_gyro_X_hjorth_complexity(),t_body_gyro_X_sec_diff_mean(),t_body_gyro_X_sec_diff_max(),...,t_body_gyro_jerk_Mag_first_diff_std(),t_body_gyro_jerk_Mag_wavelet_cA_mean(),t_body_gyro_jerk_Mag_wavelet_cA_std(),t_body_gyro_jerk_Mag_wavelet_cA_Energy(),t_body_gyro_jerk_Mag_wavelet_Entropy_A(),t_body_gyro_jerk_Mag_wavelet_cD_mean(),t_body_gyro_jerk_Mag_wavelet_cD_std(),t_body_gyro_jerk_Mag_wavelet_cD_Energy(),t_body_gyro_jerk_Mag_wavelet_Entropy_D(),measurement_id
0,2VSP,1.002650,-1.075097e+05,-0.000059,0.216099,0.754624,0.323896,2.439451,0.067972,5.778713,...,7.972038,10.147031,18.883790,1.378725e+07,1.167421e+08,0.021184,5.664321,9.625816e+05,6.456324e+06,05205dc3-9c82-4b6b-921e-b6c2edd2c993
1,2WLT,1.016804,-4.997979e+05,-0.005478,0.684763,4.204157,0.392170,2.252779,0.324338,20.884087,...,21.441469,41.318250,42.222093,1.047006e+08,1.000636e+09,0.017429,17.923587,9.637979e+06,7.701527e+07,aba31c29-79ef-4221-9412-156538a2fd4e
2,2ZX3,1.001110,-1.335227e+06,0.000793,0.139590,0.171924,0.414643,2.156222,0.039527,2.625508,...,5.278354,5.233988,11.835444,5.024339e+06,3.873472e+07,0.016609,3.842511,4.429698e+05,2.659358e+06,6cab466b-3c95-4cc0-ba1b-8650f5a12504
3,2WH7,1.000122,-2.462591e+05,-0.000014,0.043390,0.030810,0.322568,3.439114,0.012659,3.223670,...,2.086422,1.487288,3.971671,5.396218e+05,3.635382e+06,0.005732,1.463915,6.429668e+04,3.512790e+05,ed560c25-e5c5-4dba-82c7-3fc18c248ce4
4,2WH7,1.000468,-7.972110e+05,0.000003,0.081894,0.118952,0.309429,3.487643,0.031890,2.545157,...,3.672300,3.638508,7.491964,2.081187e+06,1.426769e+07,-0.008545,2.772051,2.305455e+05,1.182145e+06,f96752b5-850e-4a5a-a74a-69ab4893b6aa


In [42]:
a_real=glob.glob("ancillary_data/smartwatch_gyroscope/*.csv")
a_real_gyro=[]
num=[]
for i in a_real:
    data=pd.read_csv(i)
    a=data.groupby('device_id').agg({'x':'var','y':'count'}).reset_index()
    deviceid=a.loc[a.x.idxmax(),'device_id']
    if int(a.loc[a.device_id==deviceid,'y'])<=data.shape[0]*0.2:
        deviceid=a.loc[a.x.idxmin(),'device_id']
    data=data[data.device_id==deviceid].reset_index(drop=True)
    num.append(data.shape[0])
    if data.shape[0]>=2000:
        a_real_gyro.append(i)
#pd.DataFrame({'num':num}).sort_values(by='num')

In [43]:
len(a_real_gyro),len(a_real)

(426, 428)

In [44]:
# smartwatch_accelerometer
start_time = time.time()
num_cores = multiprocessing.cpu_count()
result=Parallel(n_jobs=num_cores)(delayed(Dataset_Generation_PipeLine_smartwatch_gyro)(i) for i in a_real_gyro)
print("--- %s Mins ---" % ((time.time() - start_time)/60))
#8.7881 Mins

--- 8.788118120034536 Mins ---


In [45]:
df_ancillary=pd.DataFrame(result)
df_ancillary.columns=['device_id_gyro']+fn_gyro
df_ancillary['measurement_id']=[item[len('ancillary_data/smartwatch_gyroscope/'):-4] for item in a_real_gyro]
print(df_ancillary.shape)
df_ancillary.head()

(426, 170)


,device_id_gyro,t_body_gyro_X_katz(),t_body_gyro_X_coeff_var(),t_body_gyro_X_slope_mean(),t_body_gyro_X_slope_std(),t_body_gyro_X_hjorth_activity(),t_body_gyro_X_hjorth_mobility(),t_body_gyro_X_hjorth_complexity(),t_body_gyro_X_sec_diff_mean(),t_body_gyro_X_sec_diff_max(),...,t_body_gyro_jerk_Mag_first_diff_std(),t_body_gyro_jerk_Mag_wavelet_cA_mean(),t_body_gyro_jerk_Mag_wavelet_cA_std(),t_body_gyro_jerk_Mag_wavelet_cA_Energy(),t_body_gyro_jerk_Mag_wavelet_Entropy_A(),t_body_gyro_jerk_Mag_wavelet_cD_mean(),t_body_gyro_jerk_Mag_wavelet_cD_std(),t_body_gyro_jerk_Mag_wavelet_cD_Energy(),t_body_gyro_jerk_Mag_wavelet_Entropy_D(),measurement_id
0,2WLT,1.001518,-7128.906056,0.000426,0.189003,0.129512,0.561299,2.641192,0.059124,2.652827,...,7.141569,7.331720,14.415857,7.847655e+06,6.188174e+07,0.002375,5.389710,871527.436365,5.372407e+06,eab1cc17-40aa-4e1f-8e4f-64e6d5743509
1,2WH7,1.000033,-601988.101763,0.000062,0.020153,0.008715,0.302088,2.914152,0.004172,1.400709,...,0.934095,0.546636,2.072678,1.272944e+05,7.777119e+05,0.001082,0.645520,11544.192736,5.162663e+04,166ba983-209f-4639-a5a6-d6e66adeba2b
2,2WH5,1.001486,-132666.862639,-0.000288,0.164944,0.365646,0.363827,2.277880,0.035475,4.992081,...,6.952026,5.249308,17.532807,8.483394e+06,7.903403e+07,0.060075,4.866248,599844.050366,4.365862e+06,bca5e12d-9fd6-496b-ac08-9e2472d8b299
3,327T,1.000003,-397294.625718,0.000055,0.006280,0.000472,0.374125,2.306019,0.003537,0.142443,...,0.214192,0.552312,0.505713,1.682439e+04,1.089687e+04,0.001267,0.183083,1005.665506,1.967664e+03,49f80736-6b50-44a6-a77b-9b1572334a8c
4,327T,1.000147,-122124.650848,-0.000266,0.043249,0.025197,0.364137,2.267640,0.015788,0.809859,...,1.627956,2.008968,3.842988,5.641541e+05,3.044145e+06,0.010548,1.194550,42813.228949,1.397687e+05,26f49660-ce1a-4946-8f83-f88850f03ec1


In [46]:
Frame_smartwatch_gyro = training_data.append(pd.DataFrame(data = df_ancillary), ignore_index=True)
print(Frame_smartwatch_gyro.shape)
Frame_smartwatch_gyro.head()

(956, 170)


,device_id_gyro,t_body_gyro_X_katz(),t_body_gyro_X_coeff_var(),t_body_gyro_X_slope_mean(),t_body_gyro_X_slope_std(),t_body_gyro_X_hjorth_activity(),t_body_gyro_X_hjorth_mobility(),t_body_gyro_X_hjorth_complexity(),t_body_gyro_X_sec_diff_mean(),t_body_gyro_X_sec_diff_max(),...,t_body_gyro_jerk_Mag_first_diff_std(),t_body_gyro_jerk_Mag_wavelet_cA_mean(),t_body_gyro_jerk_Mag_wavelet_cA_std(),t_body_gyro_jerk_Mag_wavelet_cA_Energy(),t_body_gyro_jerk_Mag_wavelet_Entropy_A(),t_body_gyro_jerk_Mag_wavelet_cD_mean(),t_body_gyro_jerk_Mag_wavelet_cD_std(),t_body_gyro_jerk_Mag_wavelet_cD_Energy(),t_body_gyro_jerk_Mag_wavelet_Entropy_D(),measurement_id
0,2VSP,1.002650,-1.075097e+05,-0.000059,0.216099,0.754624,0.323896,2.439451,0.067972,5.778713,...,7.972038,10.147031,18.883790,1.378725e+07,1.167421e+08,0.021184,5.664321,9.625816e+05,6.456324e+06,05205dc3-9c82-4b6b-921e-b6c2edd2c993
1,2WLT,1.016804,-4.997979e+05,-0.005478,0.684763,4.204157,0.392170,2.252779,0.324338,20.884087,...,21.441469,41.318250,42.222093,1.047006e+08,1.000636e+09,0.017429,17.923587,9.637979e+06,7.701527e+07,aba31c29-79ef-4221-9412-156538a2fd4e
2,2ZX3,1.001110,-1.335227e+06,0.000793,0.139590,0.171924,0.414643,2.156222,0.039527,2.625508,...,5.278354,5.233988,11.835444,5.024339e+06,3.873472e+07,0.016609,3.842511,4.429698e+05,2.659358e+06,6cab466b-3c95-4cc0-ba1b-8650f5a12504
3,2WH7,1.000122,-2.462591e+05,-0.000014,0.043390,0.030810,0.322568,3.439114,0.012659,3.223670,...,2.086422,1.487288,3.971671,5.396218e+05,3.635382e+06,0.005732,1.463915,6.429668e+04,3.512790e+05,ed560c25-e5c5-4dba-82c7-3fc18c248ce4
4,2WH7,1.000468,-7.972110e+05,0.000003,0.081894,0.118952,0.309429,3.487643,0.031890,2.545157,...,3.672300,3.638508,7.491964,2.081187e+06,1.426769e+07,-0.008545,2.772051,2.305455e+05,1.182145e+06,f96752b5-850e-4a5a-a74a-69ab4893b6aa


In [47]:
Frame_smartwatch_acc.shape

(956, 254)

In [48]:
Frame_smartwatch=pd.merge(Frame_smartwatch_acc,Frame_smartwatch_gyro,on='measurement_id')
Frame_smartwatch=Frame_smartwatch.drop('device_id_gyro',axis=1)
print(Frame_smartwatch.shape)
Frame_smartwatch.head()

(956, 422)


,device_id_acc,t_body_acc_X_katz(),t_body_acc_X_coeff_var(),t_body_acc_X_slope_mean(),t_body_acc_X_slope_std(),t_body_acc_X_hjorth_activity(),t_body_acc_X_hjorth_mobility(),t_body_acc_X_hjorth_complexity(),t_body_acc_X_sec_diff_mean(),t_body_acc_X_sec_diff_max(),...,t_body_gyro_jerk_Mag_first_diff_max(),t_body_gyro_jerk_Mag_first_diff_std(),t_body_gyro_jerk_Mag_wavelet_cA_mean(),t_body_gyro_jerk_Mag_wavelet_cA_std(),t_body_gyro_jerk_Mag_wavelet_cA_Energy(),t_body_gyro_jerk_Mag_wavelet_Entropy_A(),t_body_gyro_jerk_Mag_wavelet_cD_mean(),t_body_gyro_jerk_Mag_wavelet_cD_std(),t_body_gyro_jerk_Mag_wavelet_cD_Energy(),t_body_gyro_jerk_Mag_wavelet_Entropy_D()
0,2VSP,1.004000,-24121.668930,-0.000043,0.274105,1.195276,0.322246,3.282729,0.122494,7.748725,...,290.197266,7.972038,10.147031,18.883790,1.378725e+07,1.167421e+08,0.021184,5.664321,9.625816e+05,6.456324e+06
1,2WLT,1.020188,58603.225645,0.000753,0.942833,5.940303,0.402234,2.863228,0.413388,24.422981,...,898.104244,21.441469,41.318250,42.222093,1.047006e+08,1.000636e+09,0.017429,17.923587,9.637979e+06,7.701527e+07
2,2ZX3,1.002798,-545517.997583,0.001156,0.228635,0.662064,0.373940,2.764917,0.088138,5.926416,...,123.214710,5.278354,5.233988,11.835444,5.024339e+06,3.873472e+07,0.016609,3.842511,4.429698e+05,2.659358e+06
3,2WH7,1.000423,-144341.044526,0.000079,0.088970,0.095432,0.330273,4.100591,0.046259,3.171407,...,166.230191,2.086422,1.487288,3.971671,5.396218e+05,3.635382e+06,0.005732,1.463915,6.429668e+04,3.512790e+05
4,2WH7,1.001493,39152.736486,0.000002,0.198279,0.312401,0.415120,3.209567,0.070423,6.303599,...,123.358364,3.672300,3.638508,7.491964,2.081187e+06,1.426769e+07,-0.008545,2.772051,2.305455e+05,1.182145e+06


In [52]:
Frame_smartwatch.to_csv('realpd_comp_training_abhiroop_lastfeatures_smartwatch.csv',index=False)

### test

In [49]:
a_real=glob.glob("testing_data/smartwatch_gyroscope/*.csv")
a_real_gyro=[]
num=[]
for i in a_real:
    data=pd.read_csv(i)
    a=data.groupby('device_id').agg({'x':'var','y':'count'}).reset_index()
    deviceid=a.loc[a.x.idxmax(),'device_id']
    if int(a.loc[a.device_id==deviceid,'y'])<=data.shape[0]*0.2:
        deviceid=a.loc[a.x.idxmin(),'device_id']
    data=data[data.device_id==deviceid].reset_index(drop=True)
    num.append(data.shape[0])
    if data.shape[0]>=2000:
        a_real_gyro.append(i)

print(len(a_real_gyro),len(a_real))

# smartwatch_accelerometer
start_time = time.time()
num_cores = multiprocessing.cpu_count()
result=Parallel(n_jobs=num_cores)(delayed(Dataset_Generation_PipeLine_smartwatch_gyro)(i) for i in a_real_gyro)
print("--- %s Mins ---" % ((time.time() - start_time)/60))
#1.8061357021331788 Mins

training_data=pd.DataFrame(result)
training_data.columns=['device_id_gyro']+fn_gyro
training_data['measurement_id']=[item[len('testing_data/smartwatch_gyroscope/'):-4] for item in a_real_gyro]

Frame_smartwatch_gyro_test = training_data.copy()
print(Frame_smartwatch_gyro_test.shape)
Frame_smartwatch_gyro_test.head()

171 172
--- 4.558271133899689 Mins ---
(171, 170)


,device_id_gyro,t_body_gyro_X_katz(),t_body_gyro_X_coeff_var(),t_body_gyro_X_slope_mean(),t_body_gyro_X_slope_std(),t_body_gyro_X_hjorth_activity(),t_body_gyro_X_hjorth_mobility(),t_body_gyro_X_hjorth_complexity(),t_body_gyro_X_sec_diff_mean(),t_body_gyro_X_sec_diff_max(),...,t_body_gyro_jerk_Mag_first_diff_std(),t_body_gyro_jerk_Mag_wavelet_cA_mean(),t_body_gyro_jerk_Mag_wavelet_cA_std(),t_body_gyro_jerk_Mag_wavelet_cA_Energy(),t_body_gyro_jerk_Mag_wavelet_Entropy_A(),t_body_gyro_jerk_Mag_wavelet_cD_mean(),t_body_gyro_jerk_Mag_wavelet_cD_std(),t_body_gyro_jerk_Mag_wavelet_cD_Energy(),t_body_gyro_jerk_Mag_wavelet_Entropy_D(),measurement_id
0,2ZX6,1.011536,-1.564920e+06,-0.000162,0.530383,2.045313,0.434484,2.233903,0.273384,6.634410,...,16.764530,33.986253,34.416915,7.019002e+07,6.354244e+08,0.061358,14.301418,6.136235e+06,4.363051e+07,7860035d-c9df-44e9-ba0e-0855dfad90f4
1,2WH8,1.001157,-7.029760e+04,-0.000121,0.136866,0.230913,0.371402,2.357704,0.042840,3.599173,...,6.244261,6.485373,13.668825,6.867134e+06,5.609958e+07,-0.041733,4.458588,5.964424e+05,4.129949e+06,0c593c3f-6636-4f0f-b9b1-f489c1cd3852
2,2VSP,1.001290,-7.382567e+05,-0.000262,0.171516,0.240422,0.388788,3.528422,0.062065,6.494810,...,7.030368,6.613199,12.010681,5.640102e+06,4.349061e+07,-0.009378,5.461172,8.947944e+05,5.832560e+06,8abf7688-c6bb-488f-bb9d-c359a5f86b35
3,2VSP,1.000711,9.435552e+05,0.000855,0.114326,0.187800,0.312182,2.756888,0.034086,2.175633,...,3.939849,4.672803,9.126461,3.154032e+06,2.311465e+07,0.022121,2.850972,2.438722e+05,1.328356e+06,c4f0d5dc-c1db-4cc5-89d1-26d7fee32299
4,2WLT,1.001833,3.148517e+05,-0.000216,0.233392,0.498586,0.376490,2.347270,0.065643,16.303188,...,7.968947,9.294405,17.956460,1.226543e+07,1.184246e+08,-0.033677,5.722445,9.824908e+05,7.796013e+06,6bb1ba19-25ac-4889-8e5b-8e4215784e82


In [50]:
Frame_smartwatch_test=pd.merge(Frame_smartwatch_acc_test,Frame_smartwatch_gyro_test,on='measurement_id')
Frame_smartwatch_test=Frame_smartwatch_test.drop('device_id_gyro',axis=1)
print(Frame_smartwatch_test.shape)
Frame_smartwatch_test.head()

(171, 422)


,device_id_acc,t_body_acc_X_katz(),t_body_acc_X_coeff_var(),t_body_acc_X_slope_mean(),t_body_acc_X_slope_std(),t_body_acc_X_hjorth_activity(),t_body_acc_X_hjorth_mobility(),t_body_acc_X_hjorth_complexity(),t_body_acc_X_sec_diff_mean(),t_body_acc_X_sec_diff_max(),...,t_body_gyro_jerk_Mag_first_diff_max(),t_body_gyro_jerk_Mag_first_diff_std(),t_body_gyro_jerk_Mag_wavelet_cA_mean(),t_body_gyro_jerk_Mag_wavelet_cA_std(),t_body_gyro_jerk_Mag_wavelet_cA_Energy(),t_body_gyro_jerk_Mag_wavelet_Entropy_A(),t_body_gyro_jerk_Mag_wavelet_cD_mean(),t_body_gyro_jerk_Mag_wavelet_cD_std(),t_body_gyro_jerk_Mag_wavelet_cD_Energy(),t_body_gyro_jerk_Mag_wavelet_Entropy_D()
0,2ZX6,1.023602,88972.650089,0.002406,1.065472,6.124654,0.471469,2.507854,0.465609,55.776289,...,331.143793,16.764530,33.986253,34.416915,7.019002e+07,6.354244e+08,0.061358,14.301418,6.136235e+06,4.363051e+07
1,2WH8,1.002468,-9577.625612,0.000620,0.251940,0.497761,0.419816,2.870803,0.092780,14.599304,...,365.273467,6.244261,6.485373,13.668825,6.867134e+06,5.609958e+07,-0.041733,4.458588,5.964424e+05,4.129949e+06
2,2VSP,1.004413,-975708.091118,-0.000236,0.360364,0.575081,0.488049,3.127990,0.149894,7.094872,...,296.280466,7.030368,6.613199,12.010681,5.640102e+06,4.349061e+07,-0.009378,5.461172,8.947944e+05,5.832560e+06
3,2VSP,1.000875,67926.655283,-0.000399,0.143272,0.296430,0.294123,4.185960,0.059950,8.542191,...,117.671729,3.939849,4.672803,9.126461,3.154032e+06,2.311465e+07,0.022121,2.850972,2.438722e+05,1.328356e+06
4,2WLT,1.002326,-65635.078837,0.000161,0.274812,0.545020,0.395441,3.183180,0.100699,13.317348,...,718.572156,7.968947,9.294405,17.956460,1.226543e+07,1.184246e+08,-0.033677,5.722445,9.824908e+05,7.796013e+06


In [51]:
Frame_smartwatch_test.to_csv('realpd_comp_testing_abhiroop_lastfeatures_smartwatch.csv',index=False)

In [53]:
#smartwatch_gyroscope features
label=real_pd_training_id.append(pd.DataFrame(real_pd_ancillary_id), ignore_index=True)
df_train=pd.merge(label,Frame_smartwatch,on='measurement_id')
df_train['subject_id']=preprocessing.LabelEncoder().fit(df_train['subject_id']).transform(df_train['subject_id'])
df_train['device_id_acc']=preprocessing.LabelEncoder().fit(df_train['device_id_acc']).transform(df_train['device_id_acc'])
print(df_train.shape)
df_train.head()

(956, 426)


,measurement_id,subject_id,on_off,dyskinesia,tremor,device_id_acc,t_body_acc_X_katz(),t_body_acc_X_coeff_var(),t_body_acc_X_slope_mean(),t_body_acc_X_slope_std(),...,t_body_gyro_jerk_Mag_first_diff_max(),t_body_gyro_jerk_Mag_first_diff_std(),t_body_gyro_jerk_Mag_wavelet_cA_mean(),t_body_gyro_jerk_Mag_wavelet_cA_std(),t_body_gyro_jerk_Mag_wavelet_cA_Energy(),t_body_gyro_jerk_Mag_wavelet_Entropy_A(),t_body_gyro_jerk_Mag_wavelet_cD_mean(),t_body_gyro_jerk_Mag_wavelet_cD_std(),t_body_gyro_jerk_Mag_wavelet_cD_Energy(),t_body_gyro_jerk_Mag_wavelet_Entropy_D()
0,ee053d95-c155-400d-ae42-fe24834ad4a9,2,1.0,0.0,3.0,4,1.000188,-20526.340895,0.000181,0.059866,...,33.323622,0.786707,1.068630,1.733941,1.244598e+05,5.479447e+05,-0.013069,0.622528,11631.760520,2.506841e+04
1,ce51ee31-8553-4321-9f83-8cd3dabe2f66,2,1.0,0.0,2.0,4,1.001944,11333.770237,-0.000970,0.286324,...,210.500683,3.859154,3.362561,7.865499,2.195260e+06,1.656153e+07,-0.015964,2.842370,242387.685434,1.556088e+06
2,d3c89012-3ab9-4014-b577-61ff05e31968,2,1.0,0.0,0.0,0,1.004018,13156.478439,-0.003303,0.291894,...,206.728928,6.262726,8.760759,12.930031,5.385388e+06,4.077312e+07,-0.056107,4.843852,518059.908885,3.097808e+06
3,5c42911d-0ebd-47ba-9925-dd5ab1c0ed61,2,1.0,0.0,1.0,4,1.003646,-26119.934209,-0.002956,0.294796,...,162.089955,4.724539,7.269784,9.893913,4.522329e+06,3.139939e+07,-0.014247,3.743357,420401.804357,2.218043e+06
4,235472d5-ad2e-4c76-947e-358c9d8c1280,2,1.0,0.0,0.0,0,1.001315,-8780.809628,-0.000490,0.138259,...,110.916249,3.106651,4.291410,7.429280,2.208386e+06,1.433178e+07,0.011720,2.308537,159889.773070,7.483310e+05


In [54]:
lightgbm(df_train,'tremor')
catboost(df_train,'tremor')

lightgbm(df_train,'dyskinesia')
catboost(df_train,'dyskinesia')

lightgbm(df_train,'on_off')
catboost(df_train,'on_off')

lightgbm test_MSE tremor : 0.571
catboost test_MSE tremor : 0.584
lightgbm test_MSE dyskinesia : 0.193
catboost test_MSE dyskinesia : 0.211
lightgbm test_MSE on_off : 0.205
catboost test_MSE on_off : 0.217


In [ ]:
lightgbm test_MSE tremor : 0.571
catboost test_MSE tremor : 0.584
lightgbm test_MSE dyskinesia : 0.193
catboost test_MSE dyskinesia : 0.211
lightgbm test_MSE on_off : 0.205
catboost test_MSE on_off : 0.217

## Clinical Data

In [169]:
real_pd_demo=pd.read_csv('clinical_data/REAL-PD_Demographics.csv')
real_pd_demo['Gender']=real_pd_demo['Gender'].map({'Male':1,'Female':0})
print(real_pd_demo.shape)
real_pd_demo.head()


(22, 3)


,subject_id,Age,Gender
0,hbv002,65,1
1,hbv012,57,0
2,hbv013,68,0
3,hbv014,67,1
4,hbv016,71,1


In [170]:
real_pd_smartphone=pd.read_csv('clinical_data/REAL-PD_Smartphone_Metadata.csv')
real_pd_smartphone=real_pd_smartphone.drop(['smartphone_model','android_version','Smartphone_Location_Other'],axis=1)
real_pd_smartphone.smartphone_brand=np.where(real_pd_smartphone.smartphone_brand=='Samsung',1,0)
real_pd_smartphone.Most_Common_Smartphone_Location=np.where(real_pd_smartphone.Most_Common_Smartphone_Location=='Front pocket',1,np.where(real_pd_smartphone.Most_Common_Smartphone_Location=='Back pocket',2,3))
print(real_pd_smartphone.shape)
real_pd_smartphone.head()

(22, 3)


,subject_id,smartphone_brand,Most_Common_Smartphone_Location
0,hbv002,1,1
1,hbv012,1,1
2,hbv013,1,1
3,hbv014,1,1
4,hbv016,0,1


In [171]:
real_pd_updrs_part1_2_4=pd.read_csv('clinical_data/REAL-PD_UPDRS_Part1_2_4.csv')
print(real_pd_updrs_part1_2_4.shape)
real_pd_updrs_part1_2_4.head()


(22, 9)


,subject_id,UPDRS_PartI_Total,UPDRS_PartII_Total,UPDRS_4.1,UPDRS_4.2,UPDRS_4.3,UPDRS_4.4,UPDRS_4.5,UPDRS_4.6
0,hbv002,8,13,1,0,1,3,1,0
1,hbv012,11,4,0,0,1,0,2,2
2,hbv013,10,13,1,0,2,3,3,1
3,hbv014,9,13,0,0,1,1,3,0
4,hbv016,7,8,0,0,0,4,1,0


In [172]:
real_pd_updrs_part3=pd.read_csv('clinical_data/REAL-PD_UPDRS_Part3.csv')
real_pd_updrs_part3=real_pd_updrs_part3.fillna(1)

real_pd_updrs_part3['Rigidity']=real_pd_updrs_part3[['UPDRS_3.3 Neck','UPDRS_3.3 Right Upper Extremity','UPDRS_3.3 Left Upper Extremity','UPDRS_3.3 Right Lower Extremity','UPDRS_3.3 Left Lower Extremity']].mean(axis=1)
real_pd_updrs_part3['Tapping']=real_pd_updrs_part3[['UPDRS_3.4 Right Hand','UPDRS_3.4 Left Hand','UPDRS_3.7 Right Foot','UPDRS_3.7 Left Foot']].mean(axis=1)
real_pd_updrs_part3['Body Tremor']=real_pd_updrs_part3[['UPDRS_3.15 Right Hand','UPDRS_3.15 Left Hand','UPDRS_3.16 Right Hand','UPDRS_3.16 Left Hand','UPDRS_3.17 Right Upper Extremity','UPDRS_3.17 Left Upper Extremity','UPDRS_3.17 Right Lower Extremity','UPDRS_3.17 Left Lower Extremity','UPDRS_3.17 Lip-Jaw']].mean(axis=1)
real_pd_updrs_part3['Movement']=real_pd_updrs_part3[['UPDRS_3.5 Right Hand','UPDRS_3.5 Left Hand','UPDRS_3.6 Right Hand','UPDRS_3.6 Left Hand']].mean(axis=1)

type1=real_pd_updrs_part3.copy()
type1.drop(['ParticipantState'],axis=1,inplace=True)
numeric_cols = list(set(type1.columns) - set(['subject_id']))
type1[numeric_cols] += 1

type2=type1.groupby('subject_id').pct_change()
type2['subject_id']=real_pd_updrs_part3['subject_id']
type2=type2.groupby('subject_id').max().reset_index().fillna(0)
type2.columns=['subject_id']+list('PerChange_'+type2.columns[1:])
real_pd_updrs_part3=pd.merge(real_pd_updrs_part3[real_pd_updrs_part3.ParticipantState=='Off'].drop(['ParticipantState'],axis=1),type2,on='subject_id')
print(real_pd_updrs_part3.shape)
real_pd_updrs_part3.head()

(22, 81)


,subject_id,UPDRS_3.1,UPDRS_3.2,UPDRS_3.3 Neck,UPDRS_3.3 Right Upper Extremity,UPDRS_3.3 Left Upper Extremity,UPDRS_3.3 Right Lower Extremity,UPDRS_3.3 Left Lower Extremity,UPDRS_3.4 Right Hand,UPDRS_3.4 Left Hand,...,PerChange_UPDRS_3.17 Left Lower Extremity,PerChange_UPDRS_3.17 Lip-Jaw,PerChange_UPDRS_3.18,PerChange_UPDRS_3.19A,PerChange_UPDRS_3.19B,PerChange_UPDRS_3.20,PerChange_Rigidity,PerChange_Tapping,PerChange_Body Tremor,PerChange_Movement
0,hbv002,3,3,0,3,3,0,0,2,2,...,0.000000,0.0,0.00,1.0,-0.5,0.0,-0.272727,-0.333333,-0.214286,0.200000
1,hbv012,1,1,0,1,0,1,0,1,2,...,-0.666667,0.0,-0.25,0.0,0.0,0.0,-0.142857,-0.100000,-0.550000,-0.181818
2,hbv013,1,2,2,3,2,0,1,3,3,...,0.000000,0.0,-0.75,1.0,-0.5,0.0,-0.384615,-0.333333,-0.583333,-0.250000
3,hbv014,2,1,1,1,2,0,0,2,3,...,0.000000,0.0,0.00,0.0,0.0,0.0,-0.111111,-0.416667,-0.166667,0.000000
4,hbv016,2,2,0,3,2,0,1,0,1,...,0.000000,0.0,0.00,0.0,0.0,0.0,-0.090909,0.166667,0.076923,0.090909


In [173]:
clinical_data=pd.merge(real_pd_demo,real_pd_smartphone,on='subject_id')
clinical_data=pd.merge(clinical_data,real_pd_updrs_part1_2_4,on='subject_id')
clinical_data=pd.merge(clinical_data,real_pd_updrs_part3,on='subject_id')
print(clinical_data.shape)
clinical_data.head()

(22, 93)


,subject_id,Age,Gender,smartphone_brand,Most_Common_Smartphone_Location,UPDRS_PartI_Total,UPDRS_PartII_Total,UPDRS_4.1,UPDRS_4.2,UPDRS_4.3,...,PerChange_UPDRS_3.17 Left Lower Extremity,PerChange_UPDRS_3.17 Lip-Jaw,PerChange_UPDRS_3.18,PerChange_UPDRS_3.19A,PerChange_UPDRS_3.19B,PerChange_UPDRS_3.20,PerChange_Rigidity,PerChange_Tapping,PerChange_Body Tremor,PerChange_Movement
0,hbv002,65,1,1,1,8,13,1,0,1,...,0.000000,0.0,0.00,1.0,-0.5,0.0,-0.272727,-0.333333,-0.214286,0.200000
1,hbv012,57,0,1,1,11,4,0,0,1,...,-0.666667,0.0,-0.25,0.0,0.0,0.0,-0.142857,-0.100000,-0.550000,-0.181818
2,hbv013,68,0,1,1,10,13,1,0,2,...,0.000000,0.0,-0.75,1.0,-0.5,0.0,-0.384615,-0.333333,-0.583333,-0.250000
3,hbv014,67,1,1,1,9,13,0,0,1,...,0.000000,0.0,0.00,0.0,0.0,0.0,-0.111111,-0.416667,-0.166667,0.000000
4,hbv016,71,1,0,1,7,8,0,0,0,...,0.000000,0.0,0.00,0.0,0.0,0.0,-0.090909,0.166667,0.076923,0.090909


In [176]:
clinical_data.to_csv('realpd_clinical_preprocessed.csv',index=False)

In [178]:
#smartwatch_gyroscope features
label=real_pd_training_id.append(pd.DataFrame(real_pd_ancillary_id), ignore_index=True)
df_train=pd.merge(label,clinical_data,on='subject_id')
df_train['subject_id']=preprocessing.LabelEncoder().fit(df_train['subject_id']).transform(df_train['subject_id'])
print(df_train.shape)
df_train.head()

(1081, 97)


,measurement_id,subject_id,on_off,dyskinesia,tremor,Age,Gender,smartphone_brand,Most_Common_Smartphone_Location,UPDRS_PartI_Total,...,PerChange_UPDRS_3.17 Left Lower Extremity,PerChange_UPDRS_3.17 Lip-Jaw,PerChange_UPDRS_3.18,PerChange_UPDRS_3.19A,PerChange_UPDRS_3.19B,PerChange_UPDRS_3.20,PerChange_Rigidity,PerChange_Tapping,PerChange_Body Tremor,PerChange_Movement
0,ee053d95-c155-400d-ae42-fe24834ad4a9,2,1.0,0.0,3.0,68,0,1,1,10,...,0.0,0.0,-0.75,1.0,-0.5,0.0,-0.384615,-0.333333,-0.583333,-0.25
1,ce51ee31-8553-4321-9f83-8cd3dabe2f66,2,1.0,0.0,2.0,68,0,1,1,10,...,0.0,0.0,-0.75,1.0,-0.5,0.0,-0.384615,-0.333333,-0.583333,-0.25
2,d3c89012-3ab9-4014-b577-61ff05e31968,2,1.0,0.0,0.0,68,0,1,1,10,...,0.0,0.0,-0.75,1.0,-0.5,0.0,-0.384615,-0.333333,-0.583333,-0.25
3,5c42911d-0ebd-47ba-9925-dd5ab1c0ed61,2,1.0,0.0,1.0,68,0,1,1,10,...,0.0,0.0,-0.75,1.0,-0.5,0.0,-0.384615,-0.333333,-0.583333,-0.25
4,235472d5-ad2e-4c76-947e-358c9d8c1280,2,1.0,0.0,0.0,68,0,1,1,10,...,0.0,0.0,-0.75,1.0,-0.5,0.0,-0.384615,-0.333333,-0.583333,-0.25


In [180]:
lightgbm(df_train,'tremor')
catboost(df_train,'tremor')

lightgbm(df_train,'dyskinesia')
catboost(df_train,'dyskinesia')

lightgbm(df_train,'on_off')
catboost(df_train,'on_off')

lightgbm test_MSE tremor : 0.391
catboost test_MSE tremor : 0.391
lightgbm test_MSE dyskinesia : 0.127
catboost test_MSE dyskinesia : 0.126
lightgbm test_MSE on_off : 0.153
catboost test_MSE on_off : 0.154
